In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)

player <- read_csv("https://raw.githubusercontent.com/kwok-c/individual/refs/heads/main/players.csv")
session <- read_csv("https://raw.githubusercontent.com/kwok-c/individual/refs/heads/main/sessions.csv")

Title

introduction

Can total time played predict whether or not they are subscribed to a newsletter?

method and results

discussion 

references

In [ ]:
data <- player|>
select(played_hours, subscribe)|>
mutate(subscribe=as_factor(subscribe))

data_plot <- ggplot(data, aes(x=played_hours, fill=as_factor(subscribe)))+
geom_histogram()+
facet_grid(rows=vars(subscribe))
data_plot

In [ ]:
count<-data|>
filter(subscribe=="TRUE")
count

In [ ]:
set.seed(1)
player_split <- initial_split(data, prop = 0.75, strata = subscribe)
player_train <- training(player_split)
player_test <- testing(player_split)


knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 5) |>
  set_engine("kknn") |>
  set_mode("classification")

knn_recipe <- recipe(subscribe ~ played_hours, data = player_train)

knn_fit <- workflow() |>
  add_recipe(knn_recipe) |>
  add_model(knn_spec) |>
  fit(data = player_train)

fruit_test_predictions <- predict(knn_fit , player_test) |>
bind_cols(player_test)

fruit_prediction_accuracy <- fruit_test_predictions |>
metrics(truth = subscribe, estimate = .pred_class) 
fruit_prediction_accuracy